In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library(stringr) 

In [ ]:
doubles <- readRDS("./../Notebooks/CombinatorialPerturbations/RDSFiles/ComboEffects_doublesSameGroup.rds")
doubles <- doubles[doubles$term %ni% c("(Intercept)", "n_genes", "mt_frac", paste0("leiden", 1:9)),]
doubles[,c("std.error", "statistic", "p.value")] = NULL
doubles = reshape(doubles, idvar = "respGene", timevar = "term",  direction = "wide")
rownames(doubles) = doubles$respGene
doubles$respGene = NULL
colnames(doubles) = sapply( colnames(doubles), function(x){strsplit(x,"estimate.")[[1]][2]})


In [ ]:
combos <- readRDS("./../Notebooks/CombinatorialPerturbations/RDSFiles/ComboEffects_lm_residuals.rds")
combos <- combos[combos$term %ni% c("(Intercept)", "n_genes", "mt_frac", paste0("leiden", 1:9)),]
combos[,c("std.error", "statistic", "p.value")] = NULL
combos = reshape(combos, idvar = "respGene", timevar = "term",  direction = "wide")
rownames(combos) = combos$respGene
combos$respGene = NULL
colnames(combos) = sapply( colnames(combos), function(x){strsplit(x,"estimate.")[[1]][2]})

In [ ]:
allPred = data.frame()

for( i in c("K_0", "K_1", "K_2", "K_3", "K_4", "K_5")){       
    myR2 = round(computeR2(response = doubles[,i], prediction = 2*combos[,i] ), digits=2)
   
    
    myTemp =  data.frame("Predictedvalues" = 2*combos[,i],
                         "Observedvalues" = doubles[,i],
                         "Comb"=i,
                         "Gene" = rownames(combos),
                         "R2" = myR2,
                         "corS" = round(cor(doubles[,i], combos[,i], method="spearman"), digits=2),
                         "pearsonS" = round(cor(doubles[,i], combos[,i], method="pearson"), digits=2)
                        )
     v = prcomp(myTemp[,c("Predictedvalues", "Observedvalues")])$rotation
     myTemp$PcaFirstCompSlope = round(v[2,1]/v[1,1], digits = 2)
    
    allPred = rbind(allPred, myTemp)
      
}


In [ ]:
allPred <- allPred[allPred$Comb != "K_5",]

In [ ]:
allPred$Comb <- sapply(allPred$Comb, function(x){str_replace(x,"_","")})

In [ ]:
allPred[allPred$R2 <0, "R2"] = 0

In [ ]:
options(repr.plot.width=15, repr.plot.height=4)
pdf("Figure_5C.pdf", width = 15, height = 4)

ggplot(allPred, aes(x=Predictedvalues, y=Observedvalues)) + 
        geom_point(alpha=0.5, shape=15, color="blue") + 
        facet_wrap( ~ Comb, ncol=6)+
        geom_text(aes(label= paste0("s = ",PcaFirstCompSlope), color="red"), x=-0.25, y=0.75, size=6)+
        geom_text(aes(label= paste0("R^2 = ",R2), color="red"), x=0, y=-1, size=6)+
        geom_abline(linetype="dashed")+
        geom_abline(aes(slope = PcaFirstCompSlope,intercept=0,color="red"))+
        theme_minimal()+
        theme(axis.text = element_text(size = 15),
              axis.title = element_text(size = 15),
              strip.text = element_text(size = 15),
              legend.position='none')+
        xlim(-1,1)+ylim(-1,1)+
        ylab("Observed fold change in \n double KO cells (lg2)")+
        xlab("Sum of observed fold changes in single KO cells (lg2)")

dev.off()